# LLM Book Processing Notebook

In [1]:
!pip install openai python-dotenv requests

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("API key not found. Set OPENAI_API_KEY in .env")


In [5]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)


In [8]:
def genereer_samenvatting(titel, eisen):
    prompt = f"""
Geef een objectieve, informatieve samenvatting van het boek met de titel:
'{titel}'.

Eisen:
- {eisen}
Gebruik geen spoilers buiten de hoofdlijn en geef de kern van het verhaal weer.
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


In [9]:
titel = "The Silent Patient"
eisen = "maximaal 150 woorden, objectief, geen spoilers."
genereer_samenvatting(titel, eisen)


"'The Silent Patient', geschreven door Alex Michaelides, is een psychologische thriller die het verhaal volgt van Alicia Berenson, een succesvolle schilderes die op een dag haar man Gabriel vermoordt en daarna in totale stilte vervalt. Na de moord is zij niet in staat of bereid om te praten, waardoor haar motieven en emoties een mysterie blijven. Theo Faber, een psychotherapeut, is gefascineerd door Alicia's zaak en neemt de uitdaging aan om contact met haar te maken en de waarheid achter haar stilte te ontdekken. Terwijl hij dieper ingaat op haar verleden en zijn eigen demonen onder ogen komt, ontvouwt zich een complex web van geheimen en psychologische spanning. Het verhaal verkent thema’s als trauma, liefde en de menselijke psyche, en biedt een onthullende kijk op de gevolgen van stilte en communicatie."

In [18]:
import json
import re

def verrijk_semantiek(titel: str):
    prompt = f"""
Je bent een JSON-generator. Geef *alleen* valide JSON terug en niets anders.

Analyseer het boek '{titel}' en geef:

- thema's
- motieven
- toon/sfeer
- schrijfstijl
- karaktertypen
- narratieve structuur

Formaat (verplicht):

{{
    "themes": ["..."],
    "motifs": ["..."],
    "tone": "...",
    "writing_style": "...",
    "character_types": ["..."],
    "narrative_structure": "..."
}}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Je output is ALTIJD pure JSON. Geen tekst eromheen."},
            {"role": "user", "content": prompt}
        ]
    )

    raw = response.choices[0].message.content.strip()

    # 1. Probeer direct JSON
    try:
        return json.loads(raw)
    except:
        pass

    # 2. Zo niet: extracteer JSON-blok uit tekst
    try:
        json_text = re.search(r"\{.*\}", raw, re.DOTALL).group(0)
        return json.loads(json_text)
    except:
        print("AI-output kon niet worden geparsed:")
        print(raw)
        raise


In [19]:
verrijk_semantiek("The Silent Patient")

{'themes': ['trauma',
  'stille lijden',
  'emotionele isolatie',
  'waakzaamheid',
  'identiteit',
  'liefde en verraad'],
 'motifs': ['schilderkunst',
  'communicatie door middel van kunst',
  'de gevangenis',
  'de psycholoog-patiënt relatie'],
 'tone': 'duister en psychologisch uitdagend',
 'writing_style': 'gecombineerd met fragmentarische getuigenissen en psychologische diepgang',
 'character_types': ['de slachtoffer (Alicia)',
  'de therapeut (Theo)',
  'de antagonist (de innerlijke demonen van Alicia)',
  'de helper (de vriend en collega van Theo)'],
 'narrative_structure': 'niet-lineaire vertelling met flashbacks en verschillende perspectieven'}

In [20]:
def verwerk_boek_json(titel: str, eisen: str = None):
    data = verwerk_boek(titel, eisen)
    return json.dumps(data, indent=4, ensure_ascii=False)


In [21]:
json_blob = verwerk_boek_json("The Silent Patient")
print(json_blob)

{
    "title": "The Silent Patient",
    "summary_enriched": "'The Silent Patient' is een psychologische thriller geschreven door Alex Michaelides. Het verhaal draait om Alicia Berenson, een succesvolle schilderes die op een fatale avond haar man Gabriel vermoedt te hebben vermoord en vervolgens in een stille toestand verkeert – ze spreekt sinds die tijd geen woord meer. Haar mysterieuze zwijgen wekt de nieuwsgierigheid van Theo Faber, een psychotherapeut die vastbesloten is haar te helpen en de waarheid achter de tragedie te ontrafelen. \n\nDe roman verkent thema's zoals trauma, liefde, en de grenzen van de menselijke geest. Terwijl Theo dieper in Alicia's verleden duikt en zijn eigen persoonlijke demonen onder ogen komt, ontvouwt zich een complex web van geheimen en schokkende onthullingen. Het boek speelt met de perceptie van waarheid en de impact van stilte, waardoor het een intrigerende en gelaagde leeservaring biedt. 'The Silent Patient' is een verhaal dat lezers meeneemt in een 